In [1]:
from word2gauss import embeddings

In [2]:
embeddings.GaussianEmbedding

word2gauss.embeddings.GaussianEmbedding

In [3]:
from word2gauss import GaussianEmbedding, iter_pairs
from vocab import Vocabulary

# v = Vocabulary()
# with open('./text8.txt','r', encoding='utf-8') as corpus:
#     v.create(corpus,[(10000000,5,350)])
#
# v.save('text8_voc.gz')  #save file

# load vocab
v = Vocabulary.load('F:/project_code/text8_voc.gz')



In [4]:
embed =GaussianEmbedding(len(v),50,covariance_type='diagonal', energy_type='KL')


In [5]:
with open('F:/project_code/text8.txt','r', encoding='utf-8') as corpus:
    embed.train(iter_pairs(corpus,v),n_workers =8)

In [6]:
import numpy as np

In [7]:
import pandas as pd
example = pd.read_csv('F:/MSc DS/Project/Github_resource/hypernymysuite/hypernymysuite-master/example.tsv',sep='\t',header =0)

In [8]:
def KL_div(mui,muj, sigmai,sigmaj,K =50):
    trace_fac =0.0
    mu_diff_sq =0.0
    det_fac =1.0
    for k in range(K):
        sigma_ratio = sigmai[k]/sigmaj[k]
        trace_fac +=sigma_ratio
        mu_diff = mui[k]-muj[k]
        mu_diff_sq += mu_diff* mu_diff/sigmaj[k]
        
        det_fac *= sigma_ratio
        
        # boound def_fac
    if det_fac < 1.0e-8:
        det_fac = 1.0e-8
    elif det_fac > 1.0e8:
        det_fac = 1.0e8
        
    
#     sigma_ratio = sigmai/sigmaj
#     det_fac = K*np.log(sigma_ratio)
#     trace_fac = K* sigma_ratio
    
#     mu_diff_sq = 0
#     for k in range(K):
#         mu_diff = mui[k]-muj[k]
#         mu_diff_sq += mu_diff* mu_diff
        
    return 0.5*(trace_fac + mu_diff_sq - K - np.log(det_fac))

In [9]:
example

,hypo,hyper,sim,is_oov
0,1840s,1850s,0.127720,0
1,1850s,1860s,0.193895,0
2,1880s,1890s,0.051568,0
3,1890s,1900s,0.065130,0
4,1910s,1920s,0.038549,0
...,...,...,...,...
41177,zoom,india,0.001010,0
41178,zoroastrianism,monotheism,0.021217,0
41179,zucchini,food,0.010976,0
41180,zucchini,novel,0.004437,0


In [18]:
# use representativeness
def get_sim(word1, word2):
    result =  KL_div(embed.mu[v.word2id(word1)],embed.mu[v.word2id(word2)],embed.sigma[v.word2id(word1)],embed.sigma[v.word2id(word2)])
#     result =1/(1+kl)
    return result

In [19]:
m = example.shape[0]

In [20]:
for i in range(m):
    try:
        example.sim[i]= get_sim(example.hypo[i],example.hyper[i])
    except KeyError as e:
        example.is_oov[i] =1
        


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
example

,hypo,hyper,sim,is_oov
0,1840s,1850s,0.127720,1
1,1850s,1860s,0.193895,1
2,1880s,1890s,0.051568,1
3,1890s,1900s,0.065130,1
4,1910s,1920s,0.038549,1
...,...,...,...,...
41177,zoom,india,0.922758,0
41178,zoroastrianism,monotheism,1.244250,0
41179,zucchini,food,1.289799,0
41180,zucchini,novel,1.214786,0


In [22]:
test =np.array(example).tolist()
import csv
with open(r'F:/MSc DS/Project/Github_resource/hypernymysuite/hypernymysuite-master/KL_text8_diagonal.tsv','w',newline='') as f:
    tsv_w= csv.writer(f, delimiter='\t')
    tsv_w.writerow(['hypo','hyper','sim','is_oov'])
    tsv_w.writerows(test)

In [23]:
get_sim('cat','animal')

0.22819927569503579

In [16]:
get_sim('animal','cat')

0.8135515997241897